In [2]:
! pip install pomegranate

     |████████████████████████████████| 17.9 MB 45.0 MB/s 
     |████████████████████████████████| 15.7 MB 128 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from collections import Counter, defaultdict,namedtuple

# **Handling the file and converting them to lists of words and NER Tags**

**Read the Sentences file and words in a list**

In [3]:
my_str = ''
with open('/content/gdrive/MyDrive/Sequence Learning Assignments/HMM_Train_Sentences.txt','r') as myfile:
  my_str = myfile.read()

my_sentences = my_str.splitlines()

my_word_list = []
for str in my_sentences:
  my_word_list = my_word_list + str.split()

print(len(my_word_list))

38


In [22]:
my_word_list

['Bailey',
 'named',
 'Australia',
 'captain',
 'Starc',
 'player',
 'of',
 '2015',
 'World',
 'Cup',
 'Australia',
 'won',
 '2003',
 '2007',
 '2015',
 'Cups',
 'Melbourne',
 'Starc',
 'Warner',
 'knocks',
 'etched',
 'in',
 'memory',
 '2003',
 'SA',
 '2007',
 'WI',
 '2015',
 'Australia',
 'were',
 'venues',
 'Starc',
 'Warner',
 'Melbourne',
 'go',
 'as',
 'great',
 'combination']

**Read the NER file and store NER Tags in a list**

In [4]:
my_ner = ''
with open('/content/gdrive/MyDrive/Sequence Learning Assignments/HMM_Train_NER.txt','r') as myfile:
  my_ner = myfile.read()

my_ner_sent = my_ner.splitlines()

my_ner_list = []
for str in my_ner_sent:
  my_ner_list = my_ner_list + str.split()

print(len(my_ner_list))

38


In [23]:
my_ner_list

['PER',
 'O',
 'GEO',
 'O',
 'PER',
 'O',
 'O',
 'TIM',
 'O',
 'O',
 'GEO',
 'O',
 'TIM',
 'TIM',
 'TIM',
 'O',
 'GEO',
 'PER',
 'PER',
 'O',
 'O',
 'O',
 'O',
 'TIM',
 'GEO',
 'TIM',
 'GEO',
 'TIM',
 'GEO',
 'O',
 'O',
 'PER',
 'PER',
 'GEO',
 'O',
 'O',
 'O',
 'O']

In [24]:
my_ner

'PER O GEO O\nPER O O TIM O O\nGEO O TIM TIM TIM O\nGEO PER PER O O O O\nTIM GEO TIM GEO TIM GEO O O\nPER PER GEO O O O O'

# **Create start and end NER POS list**

In [5]:
start_POS_list = []
end_POS_list = []

for str in my_ner_sent:
  start_POS_list.append(str.split()[0])
  end_POS_list.append(str.split()[-1])

print(my_ner_sent)
print(start_POS_list)
print(end_POS_list)

['PER O GEO O', 'PER O O TIM O O', 'GEO O TIM TIM TIM O', 'GEO PER PER O O O O', 'TIM GEO TIM GEO TIM GEO O O', 'PER PER GEO O O O O']
['PER', 'PER', 'GEO', 'GEO', 'TIM', 'PER']
['O', 'O', 'O', 'O', 'O', 'O']


In [25]:
# Function to count POS
def POS_counts(sequences):
    
    n = Counter(sequences)
    return n

In [26]:
start_POS_count = POS_counts(start_POS_list)
start_POS_count

Counter({'GEO': 2, 'PER': 3, 'TIM': 1})

In [27]:
end_POS_count = POS_counts(end_POS_list)
end_POS_count

Counter({'O': 6})

# **Create Unigram and Bigrams for NER tags**

In [6]:
# Count of unigrams - how many times each POS is occuring
def unigram_token(sequences):
    return Counter(sequences)

In [29]:
POS_count_ug = unigram_token(my_ner_list)
POS_count_ug

Counter({'GEO': 7, 'O': 18, 'PER': 6, 'TIM': 7})

In [10]:
# Count of bigrams - how many times each pair of POS is occuring
def bigram_token(sequences):
    d = Counter(sequences)
    return d

In [11]:
range_index = [(my_ner_list[i],my_ner_list[i+1]) for i in range(0,len(my_ner_list)-2,1)]

print(my_ner_sent)
range_index

['PER O GEO O', 'PER O O TIM O O', 'GEO O TIM TIM TIM O', 'GEO PER PER O O O O', 'TIM GEO TIM GEO TIM GEO O O', 'PER PER GEO O O O O']


[('PER', 'O'),
 ('O', 'GEO'),
 ('GEO', 'O'),
 ('O', 'PER'),
 ('PER', 'O'),
 ('O', 'O'),
 ('O', 'TIM'),
 ('TIM', 'O'),
 ('O', 'O'),
 ('O', 'GEO'),
 ('GEO', 'O'),
 ('O', 'TIM'),
 ('TIM', 'TIM'),
 ('TIM', 'TIM'),
 ('TIM', 'O'),
 ('O', 'GEO'),
 ('GEO', 'PER'),
 ('PER', 'PER'),
 ('PER', 'O'),
 ('O', 'O'),
 ('O', 'O'),
 ('O', 'O'),
 ('O', 'TIM'),
 ('TIM', 'GEO'),
 ('GEO', 'TIM'),
 ('TIM', 'GEO'),
 ('GEO', 'TIM'),
 ('TIM', 'GEO'),
 ('GEO', 'O'),
 ('O', 'O'),
 ('O', 'PER'),
 ('PER', 'PER'),
 ('PER', 'GEO'),
 ('GEO', 'O'),
 ('O', 'O'),
 ('O', 'O')]

In [13]:
POS_count_bg = bigram_token(range_index)
POS_count_bg

Counter({('GEO', 'O'): 4,
         ('GEO', 'PER'): 1,
         ('GEO', 'TIM'): 2,
         ('O', 'GEO'): 3,
         ('O', 'O'): 8,
         ('O', 'PER'): 2,
         ('O', 'TIM'): 3,
         ('PER', 'GEO'): 1,
         ('PER', 'O'): 3,
         ('PER', 'PER'): 2,
         ('TIM', 'GEO'): 3,
         ('TIM', 'O'): 2,
         ('TIM', 'TIM'): 2})

# **Ner Word Count**

In [14]:
# Build a dictionary with each POS and words associated with those POS
def POS_word_count(POS, words):
    dDict = defaultdict(lambda: defaultdict(int))
    for POS, word in zip(POS, words):
        dDict[POS][word] += 1
        
    return dDict

In [15]:
POS_words_count=POS_word_count(my_ner_list,my_word_list)
POS_words_count

defaultdict(<function __main__.POS_word_count.<locals>.<lambda>>,
            {'GEO': defaultdict(int,
                         {'Australia': 3, 'Melbourne': 2, 'SA': 1, 'WI': 1}),
             'O': defaultdict(int,
                         {'Cup': 1,
                          'Cups': 1,
                          'World': 1,
                          'as': 1,
                          'captain': 1,
                          'combination': 1,
                          'etched': 1,
                          'go': 1,
                          'great': 1,
                          'in': 1,
                          'knocks': 1,
                          'memory': 1,
                          'named': 1,
                          'of': 1,
                          'player': 1,
                          'venues': 1,
                          'were': 1,
                          'won': 1}),
             'PER': defaultdict(int, {'Bailey': 1, 'Starc': 3, 'Warner': 2}),
             'TIM': defau

# **Create HiddenMarkovModel**

**Emmision Probabilities**

In [35]:
to_states = []
for POS, words in POS_words_count.items():
    total = float(sum(words.values()))
    print('------------------------------------------------------------')
    print(POS,'total',total)
    emission_prob = {word: count/total for word, count in words.items()}
    print('Emmission Probability',distribution)
    POS_emissions = DiscreteDistribution(emission_prob)
    #print('POS_emissions',POS_emissions)
    POS_state = State(POS_emissions, name=POS)
    #print('POS_state',POS_state)
    to_states.append(POS_state)

------------------------------------------------------------
PER total 6.0
Emmission Probability {'2015': 0.42857142857142855, '2003': 0.2857142857142857, '2007': 0.2857142857142857}
------------------------------------------------------------
O total 18.0
Emmission Probability {'2015': 0.42857142857142855, '2003': 0.2857142857142857, '2007': 0.2857142857142857}
------------------------------------------------------------
GEO total 7.0
Emmission Probability {'2015': 0.42857142857142855, '2003': 0.2857142857142857, '2007': 0.2857142857142857}
------------------------------------------------------------
TIM total 7.0
Emmission Probability {'2015': 0.42857142857142855, '2003': 0.2857142857142857, '2007': 0.2857142857142857}


In [45]:
print(len(to_states))
print([state.name for state in to_states])
print(type(to_states[0]))
print('\n\n')
print(to_states[0])

4
['PER', 'O', 'GEO', 'TIM']
<class 'pomegranate.base.State'>



{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "Bailey" : 0.16666666666666666,
                "Starc" : 0.5,
                "Warner" : 0.3333333333333333
            }
        ],
        "frozen" : false
    },
    "name" : "PER",
    "weight" : 1.0
}


**Calculate Start Probability**

In [37]:
start_prob={}
for ps in my_ner_list:
    start_prob[ps]=start_POS_count[ps]/POS_count_ug[ps]

print(start_prob)

{'PER': 0.5, 'O': 0.0, 'GEO': 0.2857142857142857, 'TIM': 0.14285714285714285}


**Calculate End Probability**

In [38]:
end_prob={}

for ps in my_ner_list:
    end_prob[ps]=end_POS_count[ps]/POS_count_ug[ps]

print(end_prob)

{'PER': 0.0, 'O': 0.3333333333333333, 'GEO': 0.0, 'TIM': 0.0}


**Transition Probability**

In [46]:
# Get the transition probability 
transition_prob_POS_word={}
for key in POS_count_bg.keys():
    transition_prob_POS_word[key]=POS_count_bg.get(key)/POS_count_ug[key[0]]

transition_prob_POS_word

{('GEO', 'O'): 0.5714285714285714,
 ('GEO', 'PER'): 0.14285714285714285,
 ('GEO', 'TIM'): 0.2857142857142857,
 ('O', 'GEO'): 0.16666666666666666,
 ('O', 'O'): 0.4444444444444444,
 ('O', 'PER'): 0.1111111111111111,
 ('O', 'TIM'): 0.16666666666666666,
 ('PER', 'GEO'): 0.16666666666666666,
 ('PER', 'O'): 0.5,
 ('PER', 'PER'): 0.3333333333333333,
 ('TIM', 'GEO'): 0.42857142857142855,
 ('TIM', 'O'): 0.2857142857142857,
 ('TIM', 'TIM'): 0.2857142857142857}

In [47]:
# Add Missing Probability
transition_prob_POS_word[('GEO', 'GEO')]=0
transition_prob_POS_word[('PER', 'TIM')]=0
transition_prob_POS_word[('TIM', 'PER')]=0

transition_prob_POS_word

{('GEO', 'GEO'): 0,
 ('GEO', 'O'): 0.5714285714285714,
 ('GEO', 'PER'): 0.14285714285714285,
 ('GEO', 'TIM'): 0.2857142857142857,
 ('O', 'GEO'): 0.16666666666666666,
 ('O', 'O'): 0.4444444444444444,
 ('O', 'PER'): 0.1111111111111111,
 ('O', 'TIM'): 0.16666666666666666,
 ('PER', 'GEO'): 0.16666666666666666,
 ('PER', 'O'): 0.5,
 ('PER', 'PER'): 0.3333333333333333,
 ('PER', 'TIM'): 0,
 ('TIM', 'GEO'): 0.42857142857142855,
 ('TIM', 'O'): 0.2857142857142857,
 ('TIM', 'PER'): 0,
 ('TIM', 'TIM'): 0.2857142857142857}

In [48]:
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

In [49]:
# Start Start and End Transitions

for POS_state in to_states :
    hmm_model.add_transition(hmm_model.start,POS_state,start_prob[POS_state.name])
    hmm_model.add_transition(POS_state,hmm_model.end,end_prob[POS_state.name])

In [50]:
# Add transition probabilities to all POS

for POS_state in to_states :
    for next_POS_state in to_states :
        hmm_model.add_transition(POS_state,next_POS_state,transition_prob_POS_word[(POS_state.name,next_POS_state.name)])

In [51]:
hmm_model.bake()

# **Decode POS for new sentence**

In [54]:
# Decode POS for a new sentence
def POS_decoding(sentence, model):    
    _, state_path = model.viterbi(sentence)
    return [state[1].name for state in state_path[1:-1]]

In [59]:
print('Starc named 2015 Australia player.”')
POS_tags = POS_decoding(('Starc','named','2015','Australia','player'), hmm_model)
print(POS_tags)

Starc named 2015 Australia player.”
['PER', 'O', 'TIM', 'GEO', 'O']
